# ANAC


#### imports

In [1324]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

#### set seed

In [1325]:
seed = 2024
np.random.seed(seed)

#### read in data

In [1326]:
diet_csv = pd.read_csv("diet.csv").copy()
recipes_csv = pd.read_csv("recipes.csv").copy()
requests_csv = pd.read_csv("requests.csv").copy()
reviews_csv = pd.read_csv("reviews.csv").copy()

/var/folders/z0/24mzs0z565s6qpzztgbdrj780000gn/T/ipykernel_24253/3884275055.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews_csv = pd.read_csv("reviews.csv").copy()


In [1327]:
diet_csv.info() # 271907 entries
requests_csv.info() # 140195 entries
recipes_csv.info() #75604 entries
reviews_csv.info() # 140195 entries
# --> not all customers gave a review!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271907 entries, 0 to 271906
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   AuthorId  271907 non-null  object
 1   Diet      271906 non-null  object
 2   Age       271907 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 6.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140195 entries, 0 to 140194
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   AuthorId      140195 non-null  object 
 1   RecipeId      140195 non-null  int64  
 2   Time          140195 non-null  float64
 3   HighCalories  140195 non-null  float64
 4   HighProtein   140195 non-null  object 
 5   LowFat        140195 non-null  int64  
 6   LowSugar      140195 non-null  object 
 7   HighFiber     140195 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 8.6+ MB
<class 'pandas.core.frame.DataFra

### Data Cleaning
#### diet_csv

In [1328]:
#rename column
diet_csv.rename(columns= {"AuthorId" : "CustomerId"}, inplace=True)
# Change type of Diet into category
diet_csv["Diet"] = diet_csv["Diet"].astype("category")
diet_csv.dropna(subset=["Diet"], inplace=True)

#### recipes_csv

In [1329]:
# Change type for Recipe Category
recipes_csv["RecipeCategory"] = recipes_csv["RecipeCategory"].astype("category")


In [1330]:
# Add new column TotalTimeNeeded
recipes_csv["TotalTimeNeeded"] = recipes_csv["CookTime"] + recipes_csv["PrepTime"]

#drop recipeservings and yields and saturated + cholesterol + sodium + carbohydratecontent 
recipes_csv = recipes_csv.drop("RecipeServings", axis=1)
recipes_csv = recipes_csv.drop("RecipeYield", axis=1)
#recipes_csv = recipes_csv.drop("SaturatedFatContent", axis=1)
#recipes_csv = recipes_csv.drop("CholesterolContent", axis=1)
#recipes_csv = recipes_csv.drop("SodiumContent", axis=1)
#recipes_csv = recipes_csv.drop("CarbohydrateContent", axis=1)
#also drop cook n preptime n quantities n parts
#recipes_csv = recipes_csv.drop("PrepTime", axis=1)
#recipes_csv = recipes_csv.drop("CookTime", axis=1)
recipes_csv = recipes_csv.drop("RecipeIngredientQuantities", axis=1)
recipes_csv = recipes_csv.drop("RecipeIngredientParts", axis=1)

In [1331]:
requests_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140195 entries, 0 to 140194
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   AuthorId      140195 non-null  object 
 1   RecipeId      140195 non-null  int64  
 2   Time          140195 non-null  float64
 3   HighCalories  140195 non-null  float64
 4   HighProtein   140195 non-null  object 
 5   LowFat        140195 non-null  int64  
 6   LowSugar      140195 non-null  object 
 7   HighFiber     140195 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 8.6+ MB


##### requests_csv

In [1332]:
#rename column
requests_csv.rename(columns= {"AuthorId" : "CustomerId"}, inplace=True)
# Map indifferent values for HighProtein and LowSugar
#requests_csv["HighProtein"] = requests_csv["HighProtein"].map({
#    "Indifferent": 0,
#    "0": 0,
#    "1": 1,
#    "Yes": 1
#})

mapping_cal = {1: 'Yes', 0.0: 'No'}
requests_csv['HighCalories'] = requests_csv['HighCalories'].map(mapping_cal).astype('category')

requests_csv['HighProtein'] = requests_csv['HighProtein'].astype('category')

mapping_fat = {1: 'Yes', 0: 'No'}
requests_csv['LowFat'] = requests_csv['LowFat'].map(mapping_fat).astype('category')

mapping_sugar = {'1':'Yes','0': 'No', 'Indifferent':'Indifferent'}
requests_csv['LowSugar'] = requests_csv['LowSugar'].map(mapping_sugar).astype('category')

mapping_fiber = {1:'Yes',0: 'No'}
requests_csv['HighFiber'] = requests_csv['HighFiber'].map(mapping_fiber).astype('category')


In [1333]:
requests_csv

,CustomerId,RecipeId,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,2001012259B,73440,1799.950949,No,Indifferent,No,No,No
1,437641B,365718,4201.820980,No,Yes,No,Indifferent,Yes
2,1803340263D,141757,6299.861496,No,Indifferent,Yes,Indifferent,No
3,854048B,280351,19801.365796,No,Yes,Yes,No,Yes
4,2277685E,180505,5400.093457,No,Indifferent,No,No,No
...,...,...,...,...,...,...,...,...
140190,163793B,78171,1560.649725,No,Indifferent,No,No,Yes
140191,33888B,333262,1502.011466,Yes,Indifferent,Yes,No,No
140192,401942C,49200,5999.274269,No,Indifferent,No,No,Yes
140193,346866B,214815,899.523513,No,Yes,Yes,Indifferent,Yes


In [1334]:
#TODO: how to handle negative time values? -> here: take the total time value from recipes by recipeId!
request_with_recipe = pd.merge(requests_csv, recipes_csv, on="RecipeId", how="left")
request_with_recipe['Time'] = np.where(request_with_recipe['Time'] < 0, request_with_recipe['TotalTimeNeeded'], request_with_recipe['Time'])
#request_with_recipe['Time'] = np.where(request_with_recipe['Time'] < 0, 0, request_with_recipe['Time'])
request_with_recipe['Time'] = request_with_recipe['Time'].round().astype('int')
request_with_recipe.rename(columns={'Time':'MaxTime'}, inplace=True)
requests_csv = request_with_recipe[['CustomerId', 'RecipeId', 'MaxTime', 'HighCalories','HighProtein', 'LowFat', 'LowSugar', 'HighFiber']]
#request_with_recipe = request_with_recipe[['CustomerId','RecipeId', 'Time','TotalTimeNeeded']]

In [1335]:
requests_csv

,CustomerId,RecipeId,MaxTime,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,2001012259B,73440,1800,No,Indifferent,No,No,No
1,437641B,365718,4202,No,Yes,No,Indifferent,Yes
2,1803340263D,141757,6300,No,Indifferent,Yes,Indifferent,No
3,854048B,280351,19801,No,Yes,Yes,No,Yes
4,2277685E,180505,5400,No,Indifferent,No,No,No
...,...,...,...,...,...,...,...,...
140190,163793B,78171,1561,No,Indifferent,No,No,Yes
140191,33888B,333262,1502,Yes,Indifferent,Yes,No,No
140192,401942C,49200,5999,No,Indifferent,No,No,Yes
140193,346866B,214815,900,No,Yes,Yes,Indifferent,Yes


#### reviews_csv

In [1336]:
#rename column
reviews_csv.rename(columns= {"AuthorId" : "CustomerId"}, inplace=True)
# Remove NA rows and Rating column and TestSetId
reviews_csv = reviews_csv.drop("Rating", axis=1)
reviews_csv.dropna(subset=["Like"], inplace=True) # note: now the entries are reduced to 97381 entries
reviews_csv = reviews_csv.drop("TestSetId", axis=1)

In [1337]:
# convert Like type to boolean
reviews_csv["Like"] = reviews_csv["Like"].astype("int")

In [1338]:
#merge diet + review
review_with_diet = pd.merge(reviews_csv, diet_csv, on="CustomerId", how="inner") # 97381 entries
#merge diet + review + request
review_diet_with_request = pd.merge(review_with_diet, requests_csv, on=["CustomerId", "RecipeId"], how="inner") # 97381 entries


In [1339]:
review_diet_with_request

,CustomerId,RecipeId,Like,Diet,Age,MaxTime,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,1000036C,320576,0,Vegetarian,50,119,No,Indifferent,No,Indifferent,Yes
1,1000216B,189335,0,Vegetarian,78,1199,No,Yes,No,No,Yes
2,1000221A,133043,0,Vegetarian,25,362,No,Yes,No,Indifferent,Yes
3,1000221A,90537,0,Vegetarian,25,1199,No,Yes,No,No,Yes
4,1000221A,334314,0,Vegetarian,25,5400,Yes,Indifferent,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...
97376,999595E,338070,0,Vegetarian,31,3899,No,Indifferent,Yes,Indifferent,No
97377,999774A,29002,0,Vegetarian,57,2402,No,Indifferent,No,Indifferent,No
97378,999774A,159252,0,Vegetarian,57,6000,No,Yes,No,No,No
97379,999774A,1171,1,Vegetarian,57,480,Yes,Yes,No,No,No


In [1341]:
# merge 'review_with_diet' with 'recipes_csv'
df = pd.merge(review_diet_with_request, recipes_csv, on='RecipeId', how='left')
df['TimeDeviation'] = df['MaxTime'] - df['TotalTimeNeeded']
#df.drop(['TotalTimeNeeded', 'MaxTime'], axis=1, inplace=True)

In [1342]:
df

,CustomerId,RecipeId,Like,Diet,Age,MaxTime,HighCalories,HighProtein,LowFat,LowSugar,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,TotalTimeNeeded,TimeDeviation
0,1000036C,320576,0,Vegetarian,50,119,No,Indifferent,No,Indifferent,...,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,120,-1
1,1000216B,189335,0,Vegetarian,78,1199,No,Yes,No,No,...,19.2,4.5,33.9,714.2,38.0,0.8,4.3,16.3,1200,-1
2,1000221A,133043,0,Vegetarian,25,362,No,Yes,No,Indifferent,...,13.5,1.9,0.0,0.6,1.9,0.4,0.4,0.2,360,2
3,1000221A,90537,0,Vegetarian,25,1199,No,Yes,No,No,...,49.0,7.2,0.0,115.8,241.7,69.4,15.7,68.9,1200,-1
4,1000221A,334314,0,Vegetarian,25,5400,Yes,Indifferent,No,No,...,11.0,3.1,110.0,1033.0,154.1,4.5,71.1,18.1,5400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97376,999595E,338070,0,Vegetarian,31,3899,No,Indifferent,Yes,Indifferent,...,31.4,15.0,99.8,391.7,43.1,1.4,27.3,5.2,3900,-1
97377,999774A,29002,0,Vegetarian,57,2402,No,Indifferent,No,Indifferent,...,33.3,8.5,33.2,318.2,107.7,10.3,13.5,32.0,2400,2
97378,999774A,159252,0,Vegetarian,57,6000,No,Yes,No,No,...,0.6,0.1,0.0,6.8,240.4,7.0,229.8,1.1,6000,0
97379,999774A,1171,1,Vegetarian,57,480,Yes,Yes,No,No,...,19.8,12.2,68.3,247.4,87.5,4.0,57.8,18.2,480,0


In [1343]:
import statsmodels.api as sm

X = df.drop(columns=["Like", "Name", "CustomerId", "RecipeId", "Diet_Vegan", "RecipeCategory_Other", "PrepTime", "CarbohydrateContent", "Calories", "SaturatedFatContent", "CookTime", "MaxTime"]).copy()
y = df['Like']

KeyError: "['Diet_Vegan', 'RecipeCategory_Other'] not found in axis"

In [ ]:
#Test for multicollinearity using VIF test
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

for index, variable_name in enumerate(X.columns):
    if variable_name == "const": 
        continue
    print(f"VIF for variable {variable_name} is {vif(X, index)}")
    

In [ ]:
# split data into learning and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
  train_test_split(X, y,
                   test_size=0.3, 
                   shuffle=True,
                   random_state=3)

In [ ]:
#fit model
model_sm = sm.Logit(y_train, sm.add_constant(X_train))
results = model_sm.fit()
print(results.summary())
# McFadden Ratio
print("McFadden Ratio",results.prsquared)

In [ ]:
# Extract the p-values for each variable
p_values = results.pvalues

# Filter variables with p-values greater than 0.05
variables_to_print = p_values[p_values > 0.1].index

# Print the variables with p-values greater than 0.05
print("Variables with p-values > 0.05:")
print(variables_to_print)

In [ ]:
# Specify the restriction for PrepTime (assuming "PrepTime" is the column name in X)
wald_test_result_prep_time = results.wald_test("(ProteinContent = 0)", scalar=True)

# Print results for PrepTime
print(f"Test statistic (chi^2_{int(wald_test_result_prep_time.df_denom)}-distributed): {wald_test_result_prep_time.statistic}")
print(f"P-value of the statistic: {wald_test_result_prep_time.pvalue}")

In [ ]:
# Predict on the test set
X_test_with_const = sm.add_constant(X_test)  # Add constant to the test set
predictions = results.predict(X_test_with_const)
y_pred_test = (predictions > 0.5).astype(int)

# Compare predictions to actual labels
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})

# Print the comparison
print(comparison_df)


In [ ]:
from sklearn.metrics import confusion_matrix, balanced_accuracy_score

# build confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
correct_predictions = (y_test == y_pred_test).sum()
total_predictions = len(y_test)
accuracy = correct_predictions / total_predictions

# Calculate Sensitivity and Specificity
sensitivity = confusion_matrix[1, 1] / (confusion_matrix[1, 1] + confusion_matrix[1, 0])
specificity = confusion_matrix[0, 0] / (confusion_matrix[0, 0] + confusion_matrix[0, 1])



# Alternatively, you can use scikit-learn's balanced_accuracy_score function
balanced_accuracy_sklearn = balanced_accuracy_score(y_test, y_pred_test)

print("Accuracy =", "{:.3f}".format(accuracy))
print("Balanced Accuracy:", balanced_accuracy_sklearn)

In [ ]:
# build confusion matrix
confusion_matrix = pd.crosstab(y_test, y_pred_test)
confusion_matrix


#### 1. Logistic Regression
#### 2. Decision Trees
#### 3. Random Forest
#### 4. Naive bayes
#### 5. Gradient Boosting

#### 1. Data Cleaning (missing values, merge tables)
#### 2. Set dataset into training & testing sets + download testing set
#### 3. modeling
#### 4. Model training: train model on training dataset
#### 5. Model evaluation (performance)
#### 6. prediction on the unseen data